In [ ]:
# Standard library imports
import warnings
import time
from pprint import pprint

# Third-party library imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import cupy as cp
import tqdm
from scipy.interpolate import interp1d
import torchaudio
import torchaudio.functional as AF
import torchaudio.transforms as AT

# Jupyter notebook specific
%matplotlib inline

# Suppress warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

# Local imports - core functionality
from coredldev.dataset import CoReDataset
from coredldev.utilites.pipeline import pipeline

# Local imports - finders and sources
from coredldev.finders.distance_scaling.h5 import h5Finder
from coredldev.sources.distance_scaling.h5 import h5Source

# Local imports - preprocessing steps
from coredldev.preprocessing.raw_postmerger.detector_angle_mixing import DetectorAngleMixing
from coredldev.preprocessing.raw_postmerger.fast_detector_angle_mixing import detector_angle_mixing as fastdam
from coredldev.preprocessing.raw_postmerger.distance_scale import distance_scale
from coredldev.preprocessing.raw_postmerger.time_shift import time_shift
from coredldev.preprocessing.to_tensor import to_tensor_clean
from coredldev.preprocessing.ligo_noise.inject_noise import NoiseInjection1D as noise_injection_1d
from coredldev.preprocessing.wavelet_transforms.morlet import MorletWaveletTransform
from coredldev.preprocessing.whiten import TimeSeriesWhitener
from pycbc.types import TimeSeries

# Import PyCBC libraries
import pycbc.noise
import pycbc.psd
import pycbc.filter

import scipy
from scipy.signal import welch

from copy import deepcopy as dp

# Load frequency values for plotting
freqs = np.genfromtxt("freqs.npy")


In [ ]:
# datapoints, eosmap, remaining = h5Finder().get_datapoints()
datapoints, eosmap, remaining = h5Finder(
    shiftpercents=[0], angles=[(0, 0, 0)], distances=[20]
).get_datapoints()
source = h5Source(eos_to_index_map=eosmap)
dataset = CoReDataset(source, datapoints, lambda x: x)
transformed_dataset = CoReDataset(
    source,
    datapoints,
    pipeline(
        [
            DetectorAngleMixing(),
            distance_scale(),
            time_shift(),
            noise_injection_1d(),
            TimeSeriesWhitener(4, 2),
            to_tensor_clean(),
        ]
    ),
)
morl = MorletWaveletTransform(freqs=np.loadtxt("freqs.npy"))
fdam = DetectorAngleMixing()
ds = distance_scale()
ts = time_shift()
ni = noise_injection_1d(psd_file="CE40-asd.txt")
ttc = to_tensor_clean()
morl = MorletWaveletTransform(freqs=np.loadtxt("freqs.npy"))
# pycbcwhitener = whiten_signal(30)
tswhitener = TimeSeriesWhitener(4, 2)
data = dataset[400]
print(f"0 {data["hplus"].shape = }")
data1 = fdam(dp(data))
print(f"1 {data1["signal"].shape = }")
data2 = ds(dp(data1))
print(f"2 {data2["signal"].shape = }")
data3 = ts(dp(data2))
print(f"3 {data3["signal"].shape = }")
data4 = ni(dp(data3))
print(f"4 {data4["signal"].shape = }")
# data5 = tswhitener(dp(data4))
# print(f"5 {data5["signal"].shape = }")
print()
prenoise_signal = data3["signal"]
signal = data4["signal"]
sam_p = data4["params"]["sam_p"]

In [ ]:
signal = data4["signal"] 
sam_p = data4["params"]["sam_p"]
frequencies, asd = welch(signal, fs=1/sam_p, nperseg=20000)
print(freqs.shape, asd.shape)
fft = np.fft.rfft(signal)
fft_freqs = np.fft.rfftfreq(len(signal), d=sam_p)
print(fft.shape, fft_freqs.shape)
asd_interpolated = np.interp(fft_freqs, frequencies, asd)
print(asd_interpolated.shape)
whitened_freqs = fft / np.sqrt(asd_interpolated) /(2*np.pi*(20_000)/sam_p+1)
print(whitened_freqs.shape)
whitened_ts = np.fft.irfft(whitened_freqs)
print(whitened_ts.shape)
plt.plot(whitened_ts)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(signal)
plt.show()
analyze_signal_frequencies(signal, sam_p, " ", True)
plt.figure(figsize=(8, 6))
plt.loglog(frequencies, np.sqrt(asd))
plt.title("Amplitude Spectral Density (ASD)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("ASD (strain/√Hz)")
plt.grid(True, which="both", ls="--", alpha=0.5)
plt.show()# Compute the ASD using Welch's method
plt.loglog(frequencies, whitened_freqs)
plt.show()

In [ ]:
def plot_steps():
    # Create a proper symlog plot for signals from data1 to data5
    plt.figure(figsize=(12, 8))

    # Plot each signal with a different color and label
    plt.plot(np.arange(len(data1["signal"])), data1["signal"], label="data1 - After Detector Angle Mixing")
    plt.plot(np.arange(len(data2["signal"])), data2["signal"], label="data2 - After Distance Scaling")
    plt.plot(np.arange(len(data3["signal"])), data3["signal"], label="data3 - After Time Shifting")
    plt.plot(np.arange(len(data4["signal"])), data4["signal"], label="data4 - After Noise Injection")
    plt.plot(np.arange(len(data5["signal"])), data5["signal"], label="data5 - After Whitening")

    # Set symlog scale for y-axis with appropriate parameters
    plt.yscale('symlog', linthresh=1e-25)  # Set linear threshold appropriate for the data

    # Add grid, legend, and labels
    plt.grid(True, which="both", ls="-", alpha=0.2)
    plt.legend(loc="best")
    plt.title("Signal Comparison Across Processing Steps")
    plt.xlabel("Sample Index")
    plt.ylabel("Signal Amplitude (symlog scale)")

    # Add a zoomed inset for data5 which has much larger values
    from mpl_toolkits.axes_grid1.inset_locator import inset_axes
    axins = inset_axes(plt.gca(), width="40%", height="30%", loc="upper right")
    axins.plot(np.arange(len(data5["signal"])), data5["signal"], 'r-')
    axins.set_title("Zoomed view of whitened signal")
    axins.grid(True, alpha=0.2)

    plt.tight_layout()
    plt.show()

    # Create a second figure to compare the signals on separate subplots
    fig, axes = plt.subplots(6, 1, figsize=(12, 15), sharex=True)

    # Plot each signal on its own subplot
    axes[0].plot(np.arange(len(data1["signal"])), data1["signal"], 'b-')
    axes[0].set_title("After Detector Angle Mixing")
    # axes[0].set_yscale('symlog', linthresh=1e-25)
    axes[0].grid(True, which="both", ls="-", alpha=0.2)

    axes[1].plot(np.arange(len(data2["signal"])), data2["signal"], 'g-')
    axes[1].set_title("After Distance Scaling")
    # axes[1].set_yscale('symlog', linthresh=1e-25)
    axes[1].grid(True, which="both", ls="-", alpha=0.2)

    axes[2].plot(np.arange(len(data3["signal"])), data3["signal"], 'r-')
    axes[2].set_title("After Time Shifting")
    # axes[2].set_yscale('symlog', linthresh=1e-25)
    axes[2].grid(True, which="both", ls="-", alpha=0.2)

    axes[3].plot(np.arange(len(data4["signal"])), data4["signal"], 'c-')
    axes[3].set_title("After Noise Injection")
    # axes[3].set_yscale('symlog', linthresh=1e-25)
    axes[3].grid(True, which="both", ls="-", alpha=0.2)

    axes[4].plot(np.arange(len(data5["signal"])), data5["signal"], 'm-')
    axes[4].set_title("After Whitening")
    axes[4].grid(True, which="both", ls="-", alpha=0.2)

    axes[5].plot(np.arange(len(data5["signal"])), data5["signal"], 'm-')
    axes[5].set_title("After Whitening (Symlog)")
    axes[5].set_yscale('symlog')
    axes[5].grid(True, which="both", ls="-", alpha=0.2)

    # Add common labels
    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    plt.xlabel("Sample Index")
    plt.ylabel("Signal Amplitude")
    plt.title("Signal Processing Steps Comparison", fontsize=16, pad=20)

    plt.tight_layout()
    plt.show()
plot_steps()